In [1]:
import torch
from diff3f_copy import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors, convert_pointcloud_to_torch_pointcloud
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map

/home/aaryap/miniconda3/envs/diff3f/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 100
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
    )
    return features.cpu()

In [4]:
pipe = init_pipe(device)
dino_model = init_dino(device)

/home/aaryap/miniconda3/envs/diff3f/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aaryap/miniconda3/envs/diff3f/lib/python3.10/site-packages/diffusers/configuration_utils.py:239: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Using cache found in /home/aaryap/.cache/torch/hub/facebookresearch_dinov2_main
/home/aaryap/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/aaryap/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: Use

In [7]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [6]:
# read from ply file
source_file_path = "/home/aaryap/research_paper_implementations/L4GM-official/results/monocular/31ea9ee9c82340bc8fd8da2343f9234d/31ea9ee9c82340bc8fd8da2343f9234d_ply_frames/frame_0000.ply"
target_file_path = "/home/aaryap/research_paper_implementations/L4GM-official/results/dynamic4d/0a0bd11af4d8460f8a0f74ecf37023aa/0a0bd11af4d8460f8a0f74ecf37023aa_ply_frames/frame_0000.ply"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [8]:
f_source = compute_features(device, pipe, dino_model, source_mesh, "cow") # (N_vertices, 2048)

Rendering complete


/home/aaryap/miniconda3/envs/diff3f/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647352509/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 100/100 [07:50<00:00,  4.70s/it]


Number of missing features:  8
Copied features from nearest vertices
Time taken in mins:  7.9554786284764605


In [ ]:
f_source.shape

torch.Size([2759, 2048])

In [ ]:
f_target = compute_features(device, pipe, dino_model, target_mesh, "camel")

Rendering complete


100%|██████████| 100/100 [11:59<00:00,  7.19s/it]

Number of missing features:  2
Copied features from nearest vertices
Time taken in mins:  12.094129113356272


In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Apply functional map on features

In [ ]:
source_file_path = "meshes/cat.off"
target_file_path = "meshes/lion.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "cat")
f_target = compute_features(device, pipe, dino_model, target_mesh, "lion")

Rendering complete


100%|██████████| 100/100 [10:16<00:00,  6.17s/it]


Number of missing features:  140
Copied features from nearest vertices
Time taken in mins:  10.621053489049276
Rendering complete


100%|██████████| 100/100 [10:14<00:00,  6.14s/it]

Number of missing features:  17
Copied features from nearest vertices
Time taken in mins:  10.467332740624746


In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [ ]:
surface_map = compute_surface_map(source_file_path, target_file_path, f_source.numpy(), f_target.numpy())

mesh1 (7207, 3)
mesh2 (5000, 3)

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 2.58 s
Computing 200 eigenvectors
	Done in 1.98 s

Computing descriptors
	Normalizing descriptors

	2048 out of 2048 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 8.5e-10

Optimization :
	50 Ev on source - 50 Ev on Target
	Using 2048 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 16, nit : 13, warnflag : 0
	Done in 3.91 seconds


In [ ]:
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[surface_map.cpu().numpy()]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Part segmentation

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner
segments2 = kmeans.predict(f_target)

In [ ]:
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [ ]:
source_file_path = "meshes/posed_human.off"
target_file_path = "meshes/cat.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "naked human")
f_target = compute_features(device, pipe, dino_model, target_mesh, "cat")

Rendering complete


100%|██████████| 100/100 [10:15<00:00,  6.16s/it]


Number of missing features:  3
Copied features from nearest vertices
Time taken in mins:  10.55890489021937
Rendering complete


100%|██████████| 100/100 [10:29<00:00,  6.29s/it]

Number of missing features:  140
Copied features from nearest vertices
Time taken in mins:  10.672471817334493


In [ ]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner

segments2 = kmeans.predict(f_target)
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  